<a href="https://colab.research.google.com/github/nmit-1NT23CS128/Social-Media-Comment-Moderation/blob/main/Social_Media_Comment_Moderation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch scikit-learn pandas numpy


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
#from huggingface_hub import notebook_login
#notebook_login()


In [ ]:
import pandas as pd

data_path = "/content/drive/MyDrive/train.csv"
df = pd.read_csv(data_path)

df.head()



In [ ]:
toxicity_columns = [
    'toxic',
    'severe_toxic',
    'obscene',
    'threat',
    'insult',
    'identity_hate'
]

df['label'] = df[toxicity_columns].max(axis=1)


In [ ]:
texts = df['comment_text'].fillna("")
labels = df['label']


In [ ]:
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts,
    labels,
    test_size=0.2,
    random_state=42
)


In [ ]:
print(train_labels.value_counts())


In [ ]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


In [ ]:
train_encodings = tokenizer(
    list(train_texts),
    truncation=True,
    padding=True,
    max_length=128
)


In [ ]:
print(train_encodings.keys())


In [ ]:
print(train_encodings['input_ids'][0][:10])


In [ ]:
import torch


In [ ]:
class ToxicDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
test_encodings = tokenizer(
    list(test_texts),
    truncation=True,
    padding=True,
    max_length=128
)

train_dataset = ToxicDataset(train_encodings, train_labels)
test_dataset = ToxicDataset(test_encodings, test_labels)

In [ ]:
print(len(train_dataset))
print(len(test_dataset))


In [ ]:
!pip install sympy==1.14.0 --force-reinstall
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=2
)


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=100
)



In [ ]:
from transformers import Trainer


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)


In [ ]:
trainer.train()

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)

accuracy = accuracy_score(test_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(test_labels, preds, average='binary')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,          # increase epochs
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,          # KEY CHANGE
    eval_strategy="epoch",
    save_strategy="epoch",
    weight_decay=0.01            # regularization
)


In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Re-initialize tokenizer if it's not defined (due to potential kernel restart)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Re-initialize model if it's not defined (due to potential kernel restart)
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=2
)

model.save_pretrained("toxic_model")
tokenizer.save_pretrained("toxic_model")